In [ ]:
"""
Process

    p = multiprocessing.Process(target=func, args=[...], kwargs={...}) 类可以创建一个进程对象
        - target 进程入口函数
        - args 进程入口函数的参数列表
        - kwargs 进程入口函数参数字典
        
    p.start() 启动进程
    p.join() 等待进程执行结束
"""

from multiprocessing import Process, Queue
import time as t
from functools import cmp_to_key
import timeit as ti


def is_prime(n, result_queue):
    if n <= 1:
        result_queue.put((n, False))
        return

    for i in range(2, n):
        if n % i == 0:
            result_queue.put((n, False))
            return
        t.sleep(0.1)
    result_queue.put((n, True))


start = ti.default_timer()

queue = Queue()

for i in range(10):
    p = Process(target=is_prime, args=(i, queue))
    p.start()

r = []
while len(r) < 10:
    r.append(queue.get())

cost = ti.default_timer() - start

r.sort(key=cmp_to_key(lambda a, b: a[0] - b[0]))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
Queue

    Queue 可以在多个进程间同步数据
"""

from multiprocessing import Process, Queue

def queue_func(queue_):
    name = queue_.get()  # read from queue
    queue_.put('Hello {}'.format(name))  # write into queue


queue = Queue()

p = Process(target=queue_func, args=(queue,))
p.start()

queue.put('Alvin')  # write into queue
p.join()

r = queue.get()  # read from queue
print('* read result from queue is {}'.format(r))

In [ ]:
"""
Pip

    parent_conn, child_conn = multiprocessing.Pip() 产生一对连接
        在其中一个连接上发送数据，在另一个连接上可以接收到数据，反之亦然
    
    conn.send(...) 发送数据
    data = conn.recv() 接收数据
    
    send 和 recv 都是阻塞式操作
"""

from multiprocessing import Process, Pipe

def pip_func(conn):
    req = conn.recv()  # read from pip
    conn.send({'result': 'Hello {}'.format(req['name']), **req})  # write into pip
    conn.close()


parent_conn, child_conn = Pipe()  # create pair of pip connect

p = Process(target=pip_func, args=(child_conn,))
p.start()

parent_conn.send({'name': 'Alvin'})  # write into pip

r = parent_conn.recv()  # read from pip
parent_conn.close()

print('* read result from queue is {}'.format(r['result']))

In [ ]:
"""
Lock

    lock = multiprocessing.Lock() 是一个可以在进程间共享的锁
        lock.acquire() 占有锁
        lock.release() 释放锁
        
        with lock:     使用 with 语法
            ...
"""

from multiprocessing import Process, Lock
import timeit as ti
import time as t

def lock_func(lock, start_time):
    with lock:  # enter lock and release automated
        print('* job finish after {:.2f} sec'.format(ti.default_timer() - start))


lock = Lock()

lock.acquire()  # enter lock

start = ti.default_timer()

p = Process(target=lock_func, args=(lock, start))
p.start()

t.sleep(1)

lock.release()  # release lock

In [ ]:
"""
Share value

    value = multiprocessing.Value(c_type, val) 创建一个可在各个进程间共享的值
        - c_type 数据的 C 语言类型，参见 ctypes 模块
        - val 初始化值
        
    array = multiprocessing.Array(c_type, iterable)
        - c_type  数组元素的 C 语言类型，参见 ctypes 模块
        - iterable 数组初始化值
"""

from multiprocessing import Process, Lock, Value, Array
from ctypes import c_int, c_double, c_byte
import time as t

def lock_func(lock, nv, sa, da):
    with lock:
        print('* num value is {}'.format(nv.value))
        print('* str value is {}'.format(bytearray(sa).decode()))
        print('* arr value is {}'.format(list(da)))


lock = Lock()

lock.acquire()

n_val = Value(c_int, 0)
s_arr = Array(c_byte, 'Hello Alvin'.encode())
d_arr = Array(c_double, [0] * 10)

p = Process(target=lock_func, args=(lock, n_val, s_arr, d_arr))
p.start()


n_val.value = 100
s_arr[0:5] = b'OK!!!'
d_arr[0:] = list(range(1, 11))

t.sleep(1)

lock.release()
p.join()

In [ ]:
"""
Manager

    with multiprocessing.Manager() as manager 创建了一个管理器，在管理器管理范围内，通过管理器创建的 list 和 dict 可以在各个进程间
共享。
"""

from multiprocessing import Process, Manager

def func(d, l):
    d.update(name='Alvin')
    l.append('x')


with Manager() as manager:
    d = manager.dict()
    l = manager.list(range(10))

    p = Process(target=func, args=(d, l))

    p.start()
    p.join()

    print('* dict value {}'.format(d))
    print('* list value {}'.format(l))

In [ ]:
"""
Pool

    with multiprocessing.Pool(processes_count) as pool 用于创建一个进程池，将任务提交给进程池后，进程池会分配一个进程对任务进行处理
        - processes_count 进程池的进程总数，可以用 multiprocessing.cpu_count() 获取系统 CPU 总数
        
    ctx = multiprocessing.get_context(method=None) 用于获取一个和 multiprocessing 模块兼容的上下文对象
        - method: 'fork', 'spawn' 或 'forkserver' 中的一个
"""

import multiprocessing as mp

context = mp

# if 'forkserver' in mp.get_all_start_methods():
#     context = mp.get_context('forkserver')

number_of_cpu = mp.cpu_count()


def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


with context.Pool(processes=number_of_cpu) as pool:
    print('multiprocessing pool created, pool={}'.format(pool))

pool.join()

In [ ]:
"""
starmap
    with multiprocessing.Pool(processes=number_of_cpu) as pool:
        pool.starmap(func, iterable) 指定方法分配到不同进程执行
            - func 进程入口函数；
            - iterable 为一个迭代器，表示 func 函数如何获取参数
        
    例如有如下函数：
        def func(a, b, c):
            ...
            
    则：
        pool.starmap(func, [(a1, b1, c1), (a2, b2, c2), (a3, b3, c3)]) 表示：
        通过三个进程执行 func 函数，每个进程传递一组参数
"""

import multiprocessing as mp
import timeit as ti
import time as t

number_of_cpu = mp.cpu_count() * 2

def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


with mp.Pool(processes=number_of_cpu) as pool:
    start = ti.default_timer()
    results = pool.starmap(is_prime, zip(range(10)))

cost = ti.default_timer() - start
r = list(enumerate(results))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
apply, apply_async

    with multiprocessing.Pool(processes=number_of_cpu) as pool:
        pool.apply(func, args) 在进程池内放置一个函数调用，并等待其执行完毕，返回执行结果
            - func 进程入口函数；
            - args func 函数的参数
        
        pool.apply_async 函数和 apply 函数类似，但不等待执行完毕，返回调用句柄
        
    例如有如下函数：
        def func(a, b, c):
            ...
            
    则：
        pool.apply(func, args=(a1, b1, c1)) 表示：
        将 func 函数放到进程池中执行，传递参数 a1, b1, c1
        
    如果使用
        h = pool.apply_async(func, args=(a1, b1, c1))
        则必须获取一次结果
        result = h.get()
"""

import multiprocessing as mp
import timeit as ti
import time as t

context = mp

number_of_cpu = mp.cpu_count()


def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


results = [False] * 10
with mp.Pool(processes=number_of_cpu) as pool:
    start = ti.default_timer()

    for i in range(len(results)):
        results[i] = pool.apply(is_prime, args=(i,))

cost = ti.default_timer() - start
r = list(enumerate(results))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
map, map_async

    with multiprocessing.Pool(processes=number_of_cpu) as pool:
        pool.map(func, iterable) 在进程池内放置一个函数调用，并等待其执行完毕，返回执行结果
            - func 进程入口函数；
            - iterable 一个序列，表示 func 函数的参数
        
        pool.map_async 函数和 map 函数类似，但不等待执行完毕，返回调用句柄
        
    例如有如下函数：
        def func(a):
            ...
            
    则：
        pool.map(func, [a1, a2, a3, a4]) 表示：
        将 func 函数放到进程池中执行，依次传递参数 a1, a2, a3
        
    如果使用
        h = pool.map_async(func, [a1, a2, a3, a4])
        则必须获取一次结果
        result = h.get()
"""

import multiprocessing as mp
import timeit as ti
import time as t

number_of_cpu = mp.cpu_count()


def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


with mp.Pool(processes=number_of_cpu) as pool:
    start = ti.default_timer()
    results = pool.map(is_prime, range(10))

cost = ti.default_timer() - start
r = list(enumerate(results))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
ProcessPoolExecutor

    executor = concurrent.futures.ProcessPoolExecutor(max_workers=n_process, mp_context=None, initializer=None, initargs=())
        - max_workers workers 的数量
        - mp_context 进程上下文
        - initializer 任务初始化入口
        - initargs 任务初始化参数
        
    future = executor.submit(func, *args, **kwargs) # 提交一个任务到进程池
        - func 进程入口函数；
        - args/kwargs 进程入口函数参数；
        
    executor.shutdown(wait=True) # 关闭进程池
        - wait 是否等待所有任务结束
        
    concurrent.futures.wait(futures) # 等待所有的任务结束
    
    # 简化写法
    with ProcessPoolExecutor(n_process) as executor:
        ...
        
    executor.map(fn, *iterables, timeout=None, chunksize=1)
        - fn 进程方法入口
        - iterables 每个进程入口的参数序列
        - timeout 执行超时时间
        - chunksize 参数传递的块大小
"""

from concurrent.futures import ProcessPoolExecutor, wait
from multiprocessing import cpu_count
from time import sleep
from timeit import default_timer as timer


n_process = cpu_count()

def func(id_):
    sleep(1)
    return 'task {} was completed'.format(id_)


with ProcessPoolExecutor(n_process) as executor:
    start = timer()
    futures = [executor.submit(func, n) for n in range(n_process * 2)]
    return_futures = wait(futures, timeout=2)
    end = timer() - start

print('* there are {}/{} task was done'.format(len(return_futures.done), len(futures)))
print('* there are {}/{} task was not done'.format(len(return_futures.not_done), len(futures)))

print('* done task results: ')
for f in return_futures.done:
    print('\t{}'.format(f.result()))

print('* all tasks cost {:.2f} sec'.format(end))

print()

with ProcessPoolExecutor(n_process) as executor:
    start = timer()
    futures = executor.map(func, range(n_process * 2))

results = [r for r in futures]

end = timer() - start
print('* results are: ')
for r in results:
    print('\t{}'.format(r))

print('* all tasks cost {:.2f} sec'.format(end))